In [1]:
from tool import *
import plotly.express as px
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.drawing.image import Image
from openpyxl.styles import PatternFill, Alignment, Font, Border, Side
# import matplotlib.pyplot as plt

In [2]:
with open('D:/Data/Shipment Plan_Adjusted_new.DB', 'rb') as f:
    spdb = pickle.load(f)
with open('D:/Data/contact_window.db', 'rb') as f:
    cont = pickle.load(f)
with open('D:/Data/ODM PO.db', 'rb') as f:
    po = pickle.load(f)
    po = po[po['Cancel'] == 'N']
    po = po.loc[('OS_TCL_CN_P')]
with open('D:/Data/ODM_SR.db', 'rb') as f:
    sr_db = pickle.load(f)
with open('D:/Data/sitemap.db', 'rb') as f:
    site = pickle.load(f)

In [3]:
ver_table = spdb.groupby(['Planweek', 'From Site', 'Category', 'Created_at']).sum('SP').reset_index()
ver_table = ver_table[ver_table['From Site'].str.contains('TCL')]
ver_table.iloc[-30:, ]
# spdb.groupby(['Planweek', 'From Site', 'Category', 'Created_at']).sum('SP').reset_index().iloc[-25:, ] # 선택할 SP의 Version 목록을 출력

,Planweek,From Site,Category,Created_at,SP
711,2024-01-22(W04),OS_TCL_CN_P,Latest26w,202401260909,428758.0
742,2024-01-29(W05),OS_TCL_CN_P,Final26w,202401301524,408789.0
743,2024-01-29(W05),OS_TCL_CN_P,Final52w,202401310857,530778.0
744,2024-01-29(W05),OS_TCL_CN_P,Latest26w,202401291025,409153.0
745,2024-01-29(W05),OS_TCL_CN_P,Latest26w,202401291546,412325.0
746,2024-01-29(W05),OS_TCL_CN_P,Latest26w,202401300855,412581.0
747,2024-01-29(W05),OS_TCL_CN_P,Latest26w,202401310947,408789.0
777,2024-02-05(W06),OS_TCL_CN_P,Final26w,202402081246,430204.0
778,2024-02-05(W06),OS_TCL_CN_P,Final52w,202402081307,539755.0
779,2024-02-05(W06),OS_TCL_CN_P,Latest26w,202402050920,415119.0


In [4]:
i1 = 906

p_wk = spdb.groupby(['Planweek', 'From Site', 'Category', 'Created_at']).sum('SP').reset_index().iloc[i1, :4]['Planweek']
vendor = spdb.groupby(['Planweek', 'From Site', 'Category', 'Created_at']).sum('SP').reset_index().iloc[i1, :4]['From Site'].split('_')[1]
ver = spdb.groupby(['Planweek', 'From Site', 'Category', 'Created_at']).sum('SP').reset_index().iloc[i1, :4]['Category']

sp = spdb.merge(spdb.groupby(['Planweek', 'From Site', 'Category', 'Created_at']).sum('SP').reset_index().iloc[[i1], :4])
sp['Month'] = sp['Week Name'].apply(lambda x:f'{datetime.date.fromisoformat(x[:10]).isocalendar().year}-{get_month_from_date(datetime.date.fromisoformat(x[:10])):02d}')

sp.insert(7, 'Series', sp['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).apply(lambda x:x.split('.')[0]))
sp.insert(7, 'Inch', sp['Mapping Model.Suffix'].apply(lambda x:x[:2]))
sp = pd.merge(sp, site[['To Site', 'Subsidiary', 'Country']], how='left')

In [12]:
sp.loc[sp['Mapping Model.Suffix'].str.contains('27MC41D'), 'SP'] = sp.loc[sp['Mapping Model.Suffix'].str.contains('27MC41D'), 'SP'] * 2 # 캐나다 Costco 2 pack 모델의 개별 모니터 개수 기준으로 변경

In [18]:
sp = sp[sp['Month'].isin(['2024-03', '2024-04', '2024-05'])]

In [22]:
sp.pivot_table('SP', index=['Inch'], columns='Week Name', aggfunc=sum).to_clipboard()

In [4]:


sp.loc[sp['Division']=='PC', 'SCM'] = sp.loc[sp['Division']=='PC', 'Subsidiary'].map(cont[(cont['Div']=='PC') & (cont['Team']=='SCM')].groupby(['Subsidiary'])['Name'].max())
sp.loc[sp['Division']=='PC', 'MKT'] = sp.loc[sp['Division']=='PC', 'Subsidiary'].map(cont[(cont['Div']=='PC') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].max())
sp.loc[sp['Division']=='MNT', 'SCM'] = sp.loc[sp['Division']=='MNT', 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='SCM')].groupby(['Subsidiary'])['Name'].max())
sp.loc[sp['Division']=='MNT', 'MKT'] = sp.loc[sp['Division']=='MNT', 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].min())
us_b2c = sp.loc[sp['To Site'].str.contains('ENUS') & (sp['Mapping Model.Suffix'].apply(lambda x:x[2]) == 'B'), 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].max())
sp.loc[sp['To Site'].str.contains('ENUS') & (sp['Mapping Model.Suffix'].apply(lambda x:x[2]) == 'B'), 'MKT'] = us_b2c
sp = sp.fillna('N/A')

gap = sp.groupby(['Planweek', 'Created_at', 'From Site', 'Region', 'Category', 'Month'])['SP'].sum().unstack(['Planweek', 'Category', 'Created_at']).fillna(0).astype(int)
gap['Gap'] = gap[gap.columns[-1]] - gap[gap.columns[0]]
gap_region = gap.unstack('Month').fillna(0).astype(int)
gap_region['Sum of Gap'] = gap_region['Gap'].sum(axis=1)
gap_region.loc[('', 'Sum'), :] = gap_region.sum()
gap_region = gap_region.astype(int)

gap = sp.groupby(['Planweek', 'Created_at', 'From Site', 'Inch', 'Category', 'Month'])['SP'].sum().unstack(['Planweek', 'Category', 'Created_at']).fillna(0).astype(int)
gap['Gap'] = gap[gap.columns[-1]] - gap[gap.columns[0]]
gap_inch = gap.unstack('Month').fillna(0).astype(int)
gap_inch['Sum of Gap'] = gap_inch['Gap'].sum(axis=1)
gap_inch.loc[('', 'Sum'), :] = gap_inch.sum()
gap_inch = gap_inch.astype(int)

gap = sp.groupby(['Planweek', 'Created_at', 'From Site', 'Series', 'Category', 'Month'])['SP'].sum().unstack(['Planweek', 'Category', 'Created_at']).fillna(0).astype(int)
gap['Gap'] = gap[gap.columns[-1]] - gap[gap.columns[0]]
gap_series = gap.unstack('Month').fillna(0).astype(int)
gap_series['Sum of Gap'] = gap_series['Gap'].sum(axis=1)
gap_series.loc[('', 'Sum'), :] = gap_series.sum()
gap_series = gap_series.astype(int)

gap = sp.groupby(['Planweek', 'Created_at', 'From Site', 'Region', 'Subsidiary', 'Country', 'To Site', 'Category', 'Inch', 'Week Name', 'Month', 'SCM', 'MKT'])['SP'].sum().unstack(['Planweek', 'Category', 'Created_at']).fillna(0).astype(int)
gap['Gap'] = gap[gap.columns[-1]] - gap[gap.columns[0]]
gap_country = gap.unstack(['Month']).fillna(0).astype(int)
gap_country['Sum of Gap'] = gap_country['Gap'].sum(axis=1)
gap_country = gap_country.astype(int)

gap = sp.groupby(['Planweek', 'Created_at', 'From Site', 'Inch', 'Region', 'Subsidiary', 'Country', 'To Site', 'Category', 'Week Name', 'Month', 'SCM', 'MKT'])['SP'].sum().unstack(['Planweek', 'Category', 'Created_at']).fillna(0).astype(int)
gap['Gap'] = gap[gap.columns[-1]] - gap[gap.columns[0]]
gap_inch_country = gap.unstack(['Month']).fillna(0).astype(int)
gap_inch_country['Sum of Gap'] = gap_inch_country['Gap'].sum(axis=1)
gap_inch_country = gap_inch_country.astype(int)

c2 = sp['Created_at'] == spdb.groupby(['Planweek', 'From Site', 'Category', 'Created_at']).sum('SP').reset_index().iloc[i1, :4]['Created_at']
c3 = sp['Created_at'] == spdb.groupby(['Planweek', 'From Site', 'Category', 'Created_at']).sum('SP').reset_index().iloc[i2, :4]['Created_at']
sp1 = sp[c2].pivot_table('SP', index=['From Site', 'Division', 'Region', 'Subsidiary', 'Country', 'To Site', 'Inch', 'Series', 'Mapping Model.Suffix', 'Rep PMS', 'SCM', 'MKT'], columns='Week Name', aggfunc=sum).fillna(0).astype(int)
wk = sp1.columns[0]
while wk < sp1.columns[-1]:
    if get_weekname_from(wk, 1) not in sp1.columns:
        sp1.insert(sp1.columns.get_loc(wk) + 1, get_weekname_from(wk, 1), 0)
    wk = get_weekname_from(wk, 1)
sp2 = sp[c3].pivot_table('SP', index=['From Site', 'Division', 'Region', 'Subsidiary', 'Country', 'To Site', 'Inch', 'Series', 'Mapping Model.Suffix', 'Rep PMS', 'SCM', 'MKT'], columns='Week Name', aggfunc=sum).fillna(0).astype(int)
wk = sp2.columns[0]
while wk < sp2.columns[-1]:
    if get_weekname_from(wk, 1) not in sp2.columns:
        sp2.insert(sp2.columns.get_loc(wk) + 1, get_weekname_from(wk, 1), 0)
    wk = get_weekname_from(wk, 1)

po_stat = sp[c2]
po_stat = po_stat[po_stat['Frozen'] == 'Y'][['Region', 'Subsidiary', 'Country', 'To Site', 'Mapping Model.Suffix', 'Week Name', 'SP', 'SCM', 'MKT']]
# cont['Subsidiary'] = cont['Subsidiary'].str.split(expand=True)[0]
# po_stat['SCM'] = po_stat['Subsidiary'].map(cont[cont['Team'] == 'SCM'].groupby(['Subsidiary'])['Name'].max())
# po_stat['MKT'] = po_stat['Subsidiary'].map(cont[cont['Team'] == 'MKT'].groupby(['Subsidiary'])['Name'].max())
# us_b2b = po_stat.loc[po_stat['To Site'].str.contains('US') & (po_stat['Mapping Model.Suffix'].apply(lambda x:x[2]) == 'B'), 'Subsidiary'].map(cont[cont['Team'] == 'MKT'].groupby(['Subsidiary'])['Name'].min())
# po_stat.loc[po_stat['To Site'].str.contains('US') & (po_stat['Mapping Model.Suffix'].apply(lambda x:x[2]) == 'B'), 'MKT'] = us_b2b
po = po.reset_index()[['PO No.', 'Mapping Model.Suffix', 'To Site', 'Unit Price', 'PO1 No', 'SO No.', 'RSD', 'Final Destination', 'PO']]

# TCL 누적 선적 실적 취합
sr_db = sr_db.loc[('OS_TCL_CN_P')].reset_index()
sr_db['Week Name'] = sr_db['Ship Date'].map(get_weekname)
sr_db['To Site'] = sr_db['To Site'].replace(ship_to_map)
sr_db = pd.merge(sr_db, site[['Region', 'Subsidiary', 'To Site', 'Country']], how='left')
sr_db[['Year', 'Month']] = sr_db['Week Name'].apply(lambda x:f'{datetime.date.fromisoformat(x[:10]).isocalendar().year}-{get_month_from_date(datetime.date.fromisoformat(x[:10])):02d}').str.split('-', expand=True)
sr_db['Inch'] = sr_db['Mapping Model.Suffix'].apply(lambda x:x[:2])
sr_db['Series'] = sr_db['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).apply(lambda x:x.split('.')[0])
sr_db = sr_db.sort_values('Ship Date')
sr_db = sr_db[['Year', 'Month', 'Region', 'Country', 'Subsidiary', 'To Site', 'Inch', 'Series', 'Mapping Model.Suffix', 'PO No.', 'RSD', 'PO1 No', 'Unit Price', 'Method', 'SO No.', 'Ship Date', 'Week Name', 'Ship', 'BL No', 'Final Destination']].reset_index(drop=True)

sr = sr_db[['PO No.', 'BL No', 'Mapping Model.Suffix', 'To Site', 'Ship Date', 'Ship']].copy()
po['To Site'] = po['To Site'].replace(ship_to_map)
po = po.merge(sr, how='left')
po['Week Name'] = po['Ship Date'].map(get_weekname, na_action='ignore')
po.loc[po['Week Name'].isnull(), 'Week Name'] = po.loc[po['Week Name'].isnull(), 'RSD'].map(get_weekname)
po.loc[po['Ship Date'].isnull() & (po['Week Name'] < p_wk), 'Week Name'] = p_wk
po_stat = po_stat.merge(po, how='left').set_index(['Region', 'Subsidiary', 'Country', 'To Site', 'Mapping Model.Suffix', 'Week Name', 'SP', 'SCM', 'MKT'])
po_stat['RSD'] = po_stat['RSD'].dt.strftime('%Y-%m-%d')
po_stat['Ship Date'] = po_stat['Ship Date'].dt.strftime('%Y-%m-%d')
sr_db['RSD'] = sr_db['RSD'].dt.strftime('%Y-%m-%d')
sr_db['Ship Date'] = sr_db['Ship Date'].dt.strftime('%Y-%m-%d')

dataset = spdb[(spdb['From Site'].str.contains('TCL')) & (spdb['Category'].str.contains('Final26w'))].copy()
dataset = pd.merge(dataset, dataset.groupby(['Planweek', 'Category'])['Created_at'].max().reset_index())
dataset['Series'] = dataset['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).apply(lambda x:x.split('.')[0])
dataset['Inch'] = dataset['Mapping Model.Suffix'].apply(lambda x:x[:2])
dataset['Month'] = dataset['Week Name'].apply(lambda x:f'{datetime.date.fromisoformat(x[:10]).isocalendar().year}-{get_month_from_date(datetime.date.fromisoformat(x[:10])):02d}')
wklist = dataset['Planweek'].unique()
sp_all = dataset[dataset['Planweek'] == wklist[0]].copy()
for curr_week in wklist[1:]:
    curr_week_sp = dataset[dataset['Planweek'] == curr_week].copy()
    curr_wklist = curr_week_sp['Week Name'].unique()
    exclude_cond1 = sp_all['Planweek'] == sp_all['Planweek'].max()
    exclude_cond2 = ~sp_all['Week Name'].isin(curr_wklist)
    exclude_cond3 = sp_all['Week Name'] < sp_all['Planweek'].max()
    curr_week_sp = pd.concat([sp_all[exclude_cond1 & exclude_cond2 & exclude_cond3], curr_week_sp]).reset_index(drop=True)
    curr_week_sp['Planweek'] = curr_week
    sp_all = pd.concat([sp_all, curr_week_sp]).reset_index(drop=True)
sp_all = sp_all.sort_values('Planweek')

def make_sp_variation(dataset, segment):
    sp_by_month = dataset.groupby(['Planweek', segment]).sum('SP').reset_index()
    sp_by_month = sp_by_month.sort_values(segment)
    x_sorted = np.sort(sp_by_month['Planweek'].unique())
    fig = px.bar(sp_by_month, x='Planweek', y='SP', color=segment, title=f'{vendor} Weekly SP total by {segment}', width=1500)
    return fig.update_xaxes(categoryarray=x_sorted)

with open('D:/Data/OTD DB.bin', 'rb')  as f:
    otd_db = pickle.load(f)
div = ['MNT']
from_sites = ['OS_TCL_CN_P']
c1 = otd_db['From Site'].isin(from_sites)
c2 = otd_db['Division'].isin(div)
tcl_total = otd_db[c1].groupby('Planweek').sum(numeric_only=True)
tcl_total['OTD(%)'] = round(tcl_total['Numerator'] / tcl_total['Denominator'] * 100, 1)
mnt_total = otd_db[c2].groupby('Planweek').sum(numeric_only=True)
mnt_total['OTD(%)'] = round(mnt_total['Numerator'] / mnt_total['Denominator'] * 100, 1)

updated_time = datetime.datetime.now().strftime('%Y%m%d%H%M')
filepath_to_save = f"D:/MNT/TCL/Shipment plan/{p_wk}_{vendor}_MNT_Shipment Plan_{updated_time}.xlsx"
row_num = 1

fdest_df = pd.Series(fdest_info).reset_index()
fdest_df.columns = ['Final Destination', 'Full Name']

with open('D:/Data/TCL box.bin', 'rb') as f:
    box = pickle.load(f)

with pd.ExcelWriter(filepath_to_save) as writer:
    gap_region.to_excel(writer, sheet_name='Summary', startrow=row_num)
    table_height = gap_region.shape[0] + len(gap_region.columns[0]) + 1 + 1
    row_num = row_num + table_height
    gap_inch.to_excel(writer, sheet_name='Summary', startrow=row_num)
    table_height = gap_inch.shape[0] + len(gap_inch.columns[0]) + 1 + 1
    row_num = row_num + table_height
    gap_series.to_excel(writer, sheet_name='Summary', startrow=row_num)
    gap_country.to_excel(writer, sheet_name='Gap_by_country', startrow=1)
    gap_inch_country.to_excel(writer, sheet_name='Gap_by_inch', startrow=1)
    sp1.to_excel(writer, sheet_name='Current Plan', startrow=1, merge_cells=False)
    sp2.to_excel(writer, sheet_name='Previous Plan', startrow=1, merge_cells=False)
    po_stat.to_excel(writer, sheet_name='PO status', startrow=1)
    otd_db[c1 & c2].reset_index(drop=True).to_excel(writer, sheet_name='OTD Performance', startrow=30, index=False)
    sr_db.to_excel(writer, sheet_name='Shipment Result', startrow=1, index=False)
    fdest_df.to_excel(writer, sheet_name='FDEST Mapping', index=False)
    box.to_excel(writer, sheet_name='spare box', index=False)

In [5]:
wb = load_workbook(filepath_to_save)

color = PatternFill(fgColor='9999FF', fill_type='solid') # Blue
color2 = PatternFill(fgColor='00FF00', fill_type='solid') # Green
big_size = Font(name='맑은 고딕', size=20, bold=True, color='000099')
middle_size = Font(name='맑은 고딕', size=15, bold=True, color='000099')

border_style = Side(border_style="thin", color="000000")
border = Border(top=border_style, right=border_style, bottom=border_style, left=border_style)

In [6]:
sh = wb[wb.sheetnames[0]]
sh['A1'] = f'■ {p_wk} {vendor} Monitor Shipment Plan Review'
sh['A1'].font = big_size
min_row_num = 2
height_gap_region = len(gap_region.columns[0])
max_row_num = min_row_num + height_gap_region - 1
for row in sh.iter_rows(min_row=min_row_num, min_col=2, max_row=max_row_num, max_col=sh.max_column):
    for cell in row:
        cell.fill = color
merge_range = f'{get_column_letter(sh.max_column)}{min_row_num}:{get_column_letter(sh.max_column)}{max_row_num}'
sh.merge_cells(merge_range)
for cell in sh[merge_range][0]:
    cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

min_row_num = max_row_num + 1
for row in sh.iter_rows(min_row=min_row_num, min_col=1, max_row=min_row_num, max_col=2):
    for cell in row:
        cell.fill = color

min_row_num = min_row_num + 1
max_row_num = min_row_num + gap_region.shape[0] - 1
for row in sh.iter_rows(min_row=min_row_num, min_col=3, max_row=max_row_num, max_col=sh.max_column):
    for cell in row:
        cell.border = border
        cell.number_format = '#,##0;[RED]Δ#,##0;"-"'

min_row_num = max_row_num + 2
height_gap_inch = len(gap_inch.columns[0])
max_row_num = min_row_num + height_gap_inch - 1
for row in sh.iter_rows(min_row=min_row_num, min_col=2, max_row=max_row_num, max_col=sh.max_column):
    for cell in row:
        cell.fill = color
merge_range = f'{get_column_letter(sh.max_column)}{min_row_num}:{get_column_letter(sh.max_column)}{max_row_num}'
sh.merge_cells(merge_range)
for cell in sh[merge_range][0]:
    cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

min_row_num = max_row_num + 1
for row in sh.iter_rows(min_row=min_row_num, min_col=1, max_row=min_row_num, max_col=2):
    for cell in row:
        cell.fill = color
        
min_row_num = min_row_num + 1
max_row_num = min_row_num + gap_inch.shape[0] - 1
for row in sh.iter_rows(min_row=min_row_num, min_col=3, max_row=max_row_num, max_col=sh.max_column):
    for cell in row:
        cell.border = border
        cell.number_format = '#,##0;[RED]Δ#,##0;"-"'

min_row_num = max_row_num + 2
height_gap_series = len(gap_series.columns[0])
max_row_num = min_row_num + height_gap_series - 1
for row in sh.iter_rows(min_row=min_row_num, min_col=2, max_row=max_row_num, max_col=sh.max_column):
    for cell in row:
        cell.fill = color
merge_range = f'{get_column_letter(sh.max_column)}{min_row_num}:{get_column_letter(sh.max_column)}{max_row_num}'
sh.merge_cells(merge_range)
for cell in sh[merge_range][0]:
    cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
    
min_row_num = max_row_num + 1
for row in sh.iter_rows(min_row=min_row_num, min_col=1, max_row=min_row_num, max_col=2):
    for cell in row:
        cell.fill = color

min_row_num = min_row_num + 1
max_row_num = min_row_num + gap_series.shape[0] - 1
for row in sh.iter_rows(min_row=min_row_num, min_col=3, max_row=max_row_num, max_col=sh.max_column):
    for cell in row:
        cell.border = border
        cell.number_format = '#,##0;[RED]Δ#,##0;"-"'

sh.freeze_panes = 'C7'
sh.column_dimensions['A'].width = 13.5
sh.column_dimensions['B'].width = 11

In [7]:
sh = wb[wb.sheetnames[1]]
sh['A1'] = f'■ TCL Monitor Shipment Gap by Country'
sh['A1'].font = middle_size
sh['I1'] = 'Sub TTL'
for i in range(10, sh.max_column+1):
    col_letter = get_column_letter(i)
    sh.cell(row=1, column=i).value = f'=subtotal(9, {col_letter}7:{col_letter}{sh.max_row})'
    sh.cell(row=1, column=i).border = border
    sh.cell(row=1, column=i).number_format = '#,##0;[RED]Δ#,##0;"-"'
for row in sh.iter_rows(min_row=2, min_col=len(gap_country.index[0]) ,max_row=2+len(gap_country.columns[0])-1, max_col=sh.max_column):
    for cell in row:
        cell.fill = color
merge_range = f'{get_column_letter(sh.max_column)}{2}:{get_column_letter(sh.max_column)}{2+len(gap_country.columns[0])-1}'
sh.merge_cells(merge_range)
for cell in sh[merge_range][0]:
    cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
for row in sh.iter_rows(min_row=6, min_col=1 ,max_row=6, max_col=len(gap_country.index[0])):
    for cell in row:
        cell.fill = color
for row in sh.iter_rows(min_row=7, min_col=10, max_row=sh.max_row, max_col=sh.max_column):
    for cell in row:
        cell.border = border
        cell.number_format = '#,##0;[RED]Δ#,##0;"-"'
sh.freeze_panes = 'J7'
sh.column_dimensions['A'].width = 13.5
sh.column_dimensions['G'].width = 17.7
sh.column_dimensions['H'].width = 6.75
sh.column_dimensions['I'].width = 6.75

In [8]:
sh = wb[wb.sheetnames[2]]
sh['A1'] = f'■ TCL Monitor Shipment Gap by Inch'
sh['A1'].font = middle_size
sh['I1'] = 'Sub TTL'
for i in range(10, sh.max_column+1):
    col_letter = get_column_letter(i)
    sh.cell(row=1, column=i).value = f'=subtotal(9, {col_letter}7:{col_letter}{sh.max_row})'
    sh.cell(row=1, column=i).border = border
    sh.cell(row=1, column=i).number_format = '#,##0;[RED]Δ#,##0;"-"'
for row in sh.iter_rows(min_row=2, min_col=len(gap_inch_country.index[0]) ,max_row=2+len(gap_inch_country.columns[0])-1, max_col=sh.max_column):
    for cell in row:
        cell.fill = color
merge_range = f'{get_column_letter(sh.max_column)}{2}:{get_column_letter(sh.max_column)}{2+len(gap_country.columns[0])-1}'
sh.merge_cells(merge_range)
for cell in sh[merge_range][0]:
    cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
for row in sh.iter_rows(min_row=6, min_col=1 ,max_row=6, max_col=len(gap_inch_country.index[0])):
    for cell in row:
        cell.fill = color
for row in sh.iter_rows(min_row=7, min_col=10, max_row=sh.max_row, max_col=sh.max_column):
    for cell in row:
        cell.border = border
        cell.number_format = '#,##0;[RED]Δ#,##0;"-"'
sh.freeze_panes = 'J7'
sh.column_dimensions['A'].width = 13.5
sh.column_dimensions['G'].width = 17.7
sh.column_dimensions['H'].width = 6.75
sh.column_dimensions['I'].width = 6.75

In [9]:
sh = wb[wb.sheetnames[3]]
sh['A1'] = f'■ {p_wk} TCL Monitor Shipment Plan'
sh['A1'].font = middle_size
sh['L1'] = 'Sub TTL'
for i in range(13, sh.max_column+1):
    col_letter = get_column_letter(i)
    sh.cell(row=1, column=i).value = f'=subtotal(9, {col_letter}3:{col_letter}{sh.max_row})'
    sh.cell(row=1, column=i).border = border
    sh.cell(row=1, column=i).number_format = '#,##0;[RED]Δ#,##0;"-"'
for cell in sh[2]:
    cell.fill = color
    cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
for row in sh.iter_rows(min_row=3, min_col=13, max_row=sh.max_row, max_col=sh.max_column):
    for cell in row:
        cell.border = border
        cell.number_format = '#,##0;[RED]Δ#,##0;"-"'
sh.freeze_panes = 'M3'
sh.column_dimensions['E'].width = 13
sh.column_dimensions['F'].width = 14
sh.column_dimensions['G'].width = 5
sh.column_dimensions['I'].width = 18
sh.column_dimensions['K'].width = 6.75
sh.column_dimensions['L'].width = 6.75
sh['I1'] = 'MINI OQ Gate Completed'
oq_gate_completed_list =['22MR410-B.AEUQ', '27MR400-B.AEUQ', '22MR410-B.APDQ', '24MR400-B.AMAQ', '24MR400-B.APDQ', '27MR400-B.APDQ', '27MR400-B.ATIQ', '27MR400-B.AJLQ', '24MR400-B.AMIQ', '22MR410-B.AMAQ',
                         '22MR410-B.AUSQ', '27MR400-B.AUSQ', '22MR410-B.AWPQ', '24MR400-B.AWPQ', '27MR400-B.AWPQ', '22MR410-B.AAUQ', '22MR410-B.AJLQ', '22MR410-B.APHQ', '22MR410-B.ATIQ', '22MR410-B.ATMQ',
                         '22MR410-B.ATSQ', '22MR410-B.ATVQ', '24MR400-B.AAUQ', '24MR400-B.AHKQ', '24MR400-B.ATSQ', '27MR400-B.AAUQ', '27MR400-B.ATSQ', '27MR400-B.ATVQ', '24MR400-B.APHQ', '27MR400-B.APHQ',
                         '27MR400-B.ATMQ', '27MR400-B.AHKQ', '22MR410-B.AEKQ', '27MR400-B.AEKQ', '27MR400-B.AMIQ', '22MR410-B.AHKQ', '22MR410-B.ATRQ', '24MR400-B.ATRQ', '27MR400-B.ATRQ', '22MR410-B.AWHQ',
                         '27MR400-B.AWHQ', '22MR41A-B.ACCQ', '22MR41S-B.ACCQ', '27MR41A-B.ACCQ', '27MR41S-B.ACCQ', '22MR410.AKRQ', '27MR400.AKRQ', '27MR400-B.AMAQ', '22MR410-B.AWMQ', '24MR400-B.AMFQ',
                         '27MR400-B.AMFQ', '27MC41D-BS.ACCQ', '22MR410-B.AMFQ', 'MINI OQ Gate Completed']

for cell in sh['I']:
    if cell.value in oq_gate_completed_list:
        cell.fill = color2

In [10]:
sh = wb[wb.sheetnames[4]]
sh['A1'] = f'■ {get_weekname_from(p_wk, -1)} TCL Monitor Shipment Plan'
sh['A1'].font = middle_size
sh['L1'] = 'Sub TTL'
for i in range(13, sh.max_column+1):
    col_letter = get_column_letter(i)
    sh.cell(row=1, column=i).value = f'=subtotal(9, {col_letter}3:{col_letter}{sh.max_row})'
    sh.cell(row=1, column=i).border = border
    sh.cell(row=1, column=i).number_format = '#,##0;[RED]Δ#,##0;"-"'
for cell in sh[2]:
    cell.fill = color
    cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
for row in sh.iter_rows(min_row=3, min_col=13, max_row=sh.max_row, max_col=sh.max_column):
    for cell in row:
        cell.border = border
        cell.number_format = '#,##0;[RED]Δ#,##0;"-"'
sh.freeze_panes = 'M3'
sh.column_dimensions['E'].width = 13
sh.column_dimensions['F'].width = 14
sh.column_dimensions['G'].width = 5
sh.column_dimensions['I'].width = 18
sh.column_dimensions['K'].width = 6.75
sh.column_dimensions['L'].width = 6.75
sh['I1'] = 'MINI OQ Gate Completed'
for cell in sh['I']:
    if cell.value in oq_gate_completed_list:
        cell.fill = color2

In [11]:
sh = wb[wb.sheetnames[5]]
sh['A1'] = f'■ TCL Monitor Confirmed Shipment PO Status_{updated_time}'
sh['A1'].font = middle_size
for cell in sh[2]:
    cell.fill = color
    cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
for row in sh.iter_rows(min_row=2, min_col=1, max_row=sh.max_row, max_col=sh.max_column):
    for cell in row:
        cell.border = border
sh.freeze_panes = 'H3'
sh.column_dimensions['C'].width = 11
sh.column_dimensions['D'].width = 18
sh.column_dimensions['E'].width = 18
sh.column_dimensions['F'].width = 18
sh.column_dimensions['H'].width = 6.25
sh.column_dimensions['I'].width = 6.75
sh.column_dimensions['J'].width = 14
sh.column_dimensions['M'].width = 9.5
sh.column_dimensions['N'].width = 10.5
sh.column_dimensions['O'].width = 6.6
sh.column_dimensions['P'].width = 5.6
sh.column_dimensions['Q'].width = 12.5
sh.column_dimensions['R'].width = 10.5

In [12]:
fig = px.line(mnt_total['OTD(%)'])
fig.add_traces(px.line(tcl_total['OTD(%)'], text=tcl_total['OTD(%)'].apply(lambda x:str(x)+'%')).data[0])
fig.data[0].name = 'Monitor Total'
fig.data[1].name = 'TCL'
fig.update_traces(line=dict(color='red', dash='dash'), selector=dict(name='Monitor Total'), opacity=0.5)
fig.update_traces(line=dict(color='blue', dash='solid', width=3), selector=dict(name='TCL'), textposition='bottom left', textfont=dict(size=12))
fig.update_layout(yaxis_title='OTD(%)', height=600, width=1400)

filepath = 'D:/figure/TCL OTD weekly performance.png'
fig.write_image(filepath)
otd_plot = Image(filepath)
ws = wb['OTD Performance']
ws.freeze_panes = 'A32'
ws.add_image(otd_plot, 'A2')
ws['E30'] = 'Sub Total'
ws['F30'] = f'=subtotal(9, F32:F{ws.max_row})'
ws['G30'] = f'=subtotal(9, G32:G{ws.max_row})'
ws['H30'] = f'=G30/F30*100'
for row in ws.iter_rows(min_row=30, min_col=1, max_row=ws.max_row, max_col=ws.max_column):
    for cell in row:
        cell.border = border
for cell in ws[31]:
    cell.fill = color
    cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
for row in ws.iter_rows(min_row=32, min_col=1, max_row=ws.max_row, max_col=ws.max_column):
    for cell in row:
        cell.border = border
ws.column_dimensions['A'].width = 15
ws.column_dimensions['C'].width = 10
ws.column_dimensions['D'].width = 15
ws.column_dimensions['E'].width = 18
ws.column_dimensions['F'].width = 10
ws.column_dimensions['G'].width = 10
ws.column_dimensions['I'].width = 88

In [13]:
sh = wb[wb.sheetnames[7]]
sh['A1'] = f'■ TCL Monitor Shipment record_{updated_time}'
sh['Q1'] = 'Sub Total'
sh['R1'] = f'=subtotal(9, R3:R{sh.max_row})'
sh['A1'].font = middle_size
for cell in sh[2]:
    cell.fill = color
    cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
for row in sh.iter_rows(min_row=2, min_col=1, max_row=sh.max_row, max_col=sh.max_column):
    for cell in row:
        cell.border = border
sh.freeze_panes = 'A3'
sh.column_dimensions['D'].width = 15
sh.column_dimensions['I'].width = 18
sh.column_dimensions['J'].width = 15
sh.column_dimensions['O'].width = 12
sh.column_dimensions['P'].width = 12
sh.column_dimensions['Q'].width = 13
sh.column_dimensions['S'].width = 15

In [14]:
sh = wb[wb.sheetnames[8]]
for cell in sh[1]:
    cell.fill = color
    cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
for row in sh.iter_rows(min_row=1, min_col=1, max_row=sh.max_row, max_col=sh.max_column):
    for cell in row:
        cell.border = border
sh.column_dimensions['A'].width = 18
sh.column_dimensions['B'].width = 50

In [15]:
sh = wb[wb.sheetnames[9]]
for cell in sh[1]:
    cell.fill = color
    cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
for row in sh.iter_rows(min_row=1, min_col=1, max_row=sh.max_row, max_col=sh.max_column):
    for cell in row:
        cell.border = border
sh.column_dimensions['A'].width = 15
sh.column_dimensions['B'].width = 13
sh.column_dimensions['C'].width = 18
sh.column_dimensions['D'].width = 8

In [16]:
wb.create_sheet('SP Variation')

segments = ['Month', 'Region', 'Inch']
i = 2
for seg in segments:
    image_filepath = f'D:/figure/{vendor}_weekly_sp_history_by_{seg}.png'
    make_sp_variation(sp_all, seg).write_image(image_filepath)
    sp_plot = Image(image_filepath)
    ws = wb['SP Variation']
    ws.add_image(sp_plot, f'A{i}')
    i = i+24

image_filepath = f'D:/figure/{vendor}_multiple_plot_weekly_sp_history_by_region.png'
px.bar(sp_all, x='Planweek', y='SP', color='Month', facet_col='Region', facet_col_wrap=3, height=1000, width=1500).write_image(image_filepath)
sp_plot = Image(image_filepath)
i =2
ws.add_image(sp_plot, f'W{i}')

i = i + 48
image_filepath = f'D:/figure/{vendor}_multiple_plot_weekly_sp_history_by_inch.png'
px.bar(sp_all, x='Planweek', y='SP', color='Month', facet_col='Inch', facet_col_wrap=3, height=500, width=1500).write_image(image_filepath)
sp_plot = Image(image_filepath)
ws.add_image(sp_plot, f'W{i}')

for name in wb.sheetnames:
    wb[name].sheet_view.showGridLines = False

wb.save(filepath_to_save)
wb.close()

In [17]:
new_week = get_weekname_from(p_wk, 4)
email_body_final = f'''
Dear TCL Team,

Thanks for your coperation as always.
I would like to release thisweek({p_wk}'s newest TCL shipment plan.

This week, we have confirmed total {sp1[get_weekname_from(p_wk, 4)].sum()} units in {new_week}.
Please check each country's order and arrange production plan and proceed shipment booking without fail.

1. SP Summary

2. New confirmed Order in {new_week}

3. OTD Index : Please mark the exact reason for OTD failed orders.

4. Set PSI

'''
email_body_initial = f'''
Dear TCL Team,

Thanks for your coperation as always.
I would like to release thisweek({p_wk}'s intitial shipment plan for TCL.

As you see below, we need to confirm {sp1[get_weekname_from(p_wk, 4)].sum()} total in {new_week}.
Please review new order in {new_week} as well as future demand and commit TCL actual shipping qty correctly on GLOP based on TCL's available material conditions, production capa and etc.
If there are any troubles that cannot be commited, please let us know the reason for each country's order.


1. SP Summary

2. New orders that needs to be confirmed in {new_week}

3. OTD Index : Please mark the exact reason for OTD failed orders in the below table.

4. Recent 5-week country-wise shipment plan fluctuations: Please refer to the attached Excel file for details.

5. Set PSI : Please update us latest TCL PP so that I can make set PSI.
'''

In [18]:
if (datetime.datetime.now().strftime('%a') in ['Mon', 'Tue']) & (ver[:6] == 'Latest'):
    print(email_body_initial)
else:
    print(email_body_final)


Dear TCL Team,

Thanks for your coperation as always.
I would like to release thisweek(2024-03-04(W10)'s newest TCL shipment plan.

This week, we have confirmed total 12868 units in 2024-04-01(W14).
Please check each country's order and arrange production plan and proceed shipment booking without fail.

1. SP Summary

2. New confirmed Order in 2024-04-01(W14)

3. OTD Index : Please mark the exact reason for OTD failed orders.

4. Set PSI


